# AI Stock Watcher — Colab Runner & Patcher


This notebook sets up a clean Colab environment, uploads your two files (`multiModelStocksNightsWatch.py`, `modeltraining.txt`), and applies critical patches:

1. Fix target shape for the LSTM‑Transformer training (`create_sequences`).
2. Fix `'ModelConfig' object is not subscriptable` and add `model_dir` / `reports_dir` to the dataclass.
3. Add a `--no-mp` flag and inject a safe multiprocessing guard (`spawn`) for Colab.


In [ ]:

# ==== Install dependencies ====
!pip -q install --upgrade pip
!pip -q install numpy pandas scikit-learn statsmodels pmdarima yfinance matplotlib
# For CPU Torch in CPU runtimes; if you switch to GPU runtime, you can comment the next line.
!pip -q install torch --index-url https://download.pytorch.org/whl/cpu


In [ ]:

# ==== Verify Torch / runtime info ====
import sys, platform
try:
    import torch
    torch_ver = torch.__version__
    cuda_ok = torch.cuda.is_available()
except Exception as e:
    torch_ver = f"unavailable ({e})"
    cuda_ok = False

print("Python:", sys.version)
print("Platform:", platform.platform())
print("Torch version:", torch_ver)
print("CUDA available:", cuda_ok)


In [ ]:

# ==== (Optional) Mount Google Drive if you want to persist models/logs ====
# from google.colab import drive
# drive.mount('/content/drive')
# BASE = "/content/drive/MyDrive/ai_stock_watcher"  # adjust if you want
# %cd /content


In [ ]:

# ==== Upload your script and config ====
from google.colab import files
uploaded = files.upload()  # select: multiModelStocksNightsWatch.py, modeltraining.txt
print("Uploaded:", list(uploaded.keys()))


In [ ]:

# ==== Apply critical patches to multiModelStocksNightsWatch.py ====
import re, pathlib

code_path = "multiModelStocksNightsWatch.py"
src = pathlib.Path(code_path).read_text()

def ensure_model_paths(dataclass_src: str) -> str:
    # Inject model_dir and reports_dir into ModelConfig dataclass if not present
    if "class ModelConfig" not in dataclass_src:
        return dataclass_src
    if "model_dir:" not in dataclass_src:
        dataclass_src = re.sub(
            r"(@dataclass\s*\nclass\s+ModelConfig\s*:\s*\n(?:\s+.*\n)+?)",
            lambda m: (m.group(1) + "    model_dir: str = \"models\"\n"),
            dataclass_src, flags=re.S
        )
    if "reports_dir:" not in dataclass_src:
        dataclass_src = re.sub(
            r"(@dataclass\s*\nclass\s+ModelConfig\s*:\s*\n(?:\s+.*\n)+?)",
            lambda m: (m.group(1) + "    reports_dir: str = \"reports\"\n"),
            dataclass_src, flags=re.S
        )
    return dataclass_src

def patch_config_attribute_access(s: str) -> str:
    # Replace dict-style access on config with attribute style (common cases)
    s = s.replace("config['model_dir']", "config.model_dir")
    s = s.replace("config['reports_dir']", "config.reports_dir")
    return s

def patch_create_sequences(s: str) -> str:
    # Replace old create_sequences implementation with the corrected one.
    pattern = re.compile(
        r"def\s+create_sequences\s*\([^\)]*\):\s*[\s\S]*?return\s+np\.array\(\s*X\s*\)\s*,\s*np\.array\(\s*y\s*\)",
        re.M
    )
    repl = (
        "def create_sequences(self, X, y, sequence_length):\n"
        "    # Return: X_seqs (N, sequence_length, n_features); y_vecs (N, prediction_horizon)\n"
        "    X_seqs, y_vecs = [], []\n"
        "    for i in range(sequence_length - 1, len(X)):\n"
        "        X_seqs.append(X[i - sequence_length + 1 : i + 1, :])\n"
        "        y_vecs.append(y[i, :])\n"
        "    return np.array(X_seqs), np.array(y_vecs)"
    )
    if pattern.search(s):
        s = pattern.sub(repl, s)
    else:
        print("WARNING: create_sequences pattern not found; skipping that patch.")
    # Fix call site: ensure we pass (X, y, seq_len) not concatenated
    s = s.replace(
        "np.concatenate([X, y], axis=1),\n            self.config.sequence_length,\n            self.config.prediction_horizon\n        )",
        "X, y, self.config.sequence_length\n        )"
    )
    return s

def inject_mp_spawn_guard(s: str) -> str:
    # Add spawn guard in __main__ block for Colab
    s = s.replace(
        'if __name__ == "__main__":',
        'if __name__ == "__main__":\n    import multiprocessing as mp\n    mp.set_start_method("spawn", force=True)'
    )
    return s

def add_no_mp_flag_and_gate_pool(s: str) -> str:
    # Try to add a --no-mp flag and gate Pool usage with it.
    s = re.sub(
        r"(parser\s*=\s*argparse\.ArgumentParser\([^\)]*\)\s*\n)",
        r"\1parser.add_argument('--no-mp', action='store_true', help='Run in single process (no multiprocessing)')\n",
        s
    )
    s = re.sub(
        r"with\s+Pool\([^\)]*\)\s+as\s+(\w+)\s*:\s*\n\s*\1\.map\(train_ticker,\s*job_args\)",
        "import multiprocessing as _mp\n"
        "if not args.no_mp:\n"
        "    with _mp.Pool(processes=num_workers) as p:\n"
        "        results = p.map(train_ticker, job_args)\n"
        "else:\n"
        "    results = [train_ticker(a) for a in job_args]",
        s
    )
    s = re.sub(
        r"p\s*=\s*Pool\([^\)]*\)\s*\n\s*results\s*=\s*p\.map\(train_ticker,\s*job_args\)",
        "import multiprocessing as _mp\n"
        "if not args.no_mp:\n"
        "    p = _mp.Pool(processes=num_workers)\n"
        "    results = p.map(train_ticker, job_args)\n"
        "    p.close(); p.join()\n"
        "else:\n"
        "    results = [train_ticker(a) for a in job_args]",
        s
    )
    return s

src = ensure_model_paths(src)
src = patch_config_attribute_access(src)
src = patch_create_sequences(src)
src = inject_mp_spawn_guard(src)
src = add_no_mp_flag_and_gate_pool(src)

pathlib.Path(code_path).write_text(src)
print("Patches applied to", code_path)


In [ ]:

# ==== First run: single-process (no multiprocessing) ====
# If your script doesn't use argparse, this will be ignored harmlessly.
!python multiModelStocksNightsWatch.py --config modeltraining.txt --no-mp


In [ ]:

# ==== Optional: run with multiprocessing after confirming single-process works ====
!python multiModelStocksNightsWatch.py --config modeltraining.txt



### Output locations
- Models should be saved under `models/` (or the path from your config).
- Reports/logs (if any) under `reports/`.

If you mounted Drive, set those paths to your Drive folder inside your script or config to persist artifacts.
